## Estimating effective BAC parameters from a Gaussian channel

In [ ]:
import numpy as np
import torch
from typing import List, Tuple, Optional, Set

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

In [ ]:
from mighty_codes.torch_utils import \
    to_torch, \
    to_np

from mighty_codes.channels import \
    BinaryAsymmetricChannelModel, \
    BinaryChannelSpecification, \
    GaussianChannelModel

from scipy import optimize as opt

## Solution by sampling

In [ ]:
def get_loss(x, p_01, p_10, n_samples_per_symbol = 1_000_000):
    
    loc_1 = x[0]
    scale_1 = x[1]

    model = GaussianChannelModel(
        n_symbols=2,
        n_readout=1,
        loc_sr=to_torch(np.asarray([[0.], [loc_1]]), device, dtype),
        scale_sr=to_torch(np.asarray([[1.], [scale_1]]), device, dtype),
        device=device,
        dtype=dtype)
    
    conf_mat_ss = model.estimate_bac_channel_parameters(n_samples_per_symbol=n_samples_per_symbol)
    c_p_01 = conf_mat_ss[0, 1].item()
    c_p_10 = conf_mat_ss[1, 0].item()
    
    err = np.abs(p_01 - c_p_01) + np.abs(p_10 - c_p_10)
    print(x)
    print(err)
    return err

In [ ]:
p_01 = 0.04
p_10 = 0.10
n_samples_per_symbol = 10_000_000
f = lambda x: get_loss(x, p_01, p_10, n_samples_per_symbol)

In [ ]:
x0 = np.asarray([4.3, 2.0])
out = opt.fmin(f, x0, ftol=1e-4, xtol=1e-4, disp=True, full_output=True)

we find:

[4.36271109 2.03893602]

## Analytical

In [ ]:
from scipy.stats import norm

def get_theta(loc_1, scale_1):
    a = 1 - 1 / (scale_1 ** 2)
    b = 2 * loc_1 / (scale_1 ** 2)
    c = - (loc_1 / scale_1) ** 2 - 2 * np.log(scale_1)
    return (- b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a) 


def get_error_analytical(x, p_01, p_10):
    loc_1 = x[0]
    scale_1 = x[1]
    theta = get_theta(loc_1, scale_1)
    c_p_01 = norm.cdf(-theta, loc=0., scale=1.)
    c_p_10 = norm.cdf(theta, loc=loc_1, scale=scale_1)
    
    err = np.abs(p_01 - c_p_01) + np.abs(p_10 - c_p_10)
    return err


In [ ]:
p_01 = 0.04
p_10 = 0.10
f = lambda x: get_error_analytical(x, p_01, p_10)

In [ ]:
x0 = np.asarray([4., 2.])
out = opt.fmin(f, x0, ftol=1e-7, xtol=1e-7, disp=True, full_output=True)
print(out)